# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one (or multiple)
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using AtomsBuilder
using DFTK
using LinearAlgebra
using Unitful
using UnitfulAtomic
using PseudoPotentialData

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    # Use AtomsBuilder to setup aluminium cubic unit cell (4 Al atoms)
    # with provided lattice constant, see AtomsBase integration for details.
    unit_cell = bulk(:Al; a=7.65339u"bohr", cubic=true)
    supercell = unit_cell * (repeat, 1, 1)  # Make a supercell

    # Select standard pseudodojo pseudopotentials, construct an LDA model, discretize
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    pseudopotentials = PseudoFamily("dojo.nc.sr.lda.v0_4_1.standard.upf")
    model = model_DFT(supercell; pseudopotentials, functionals=LDA(),
                      temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

As expected we obtain the unit cell for `repeat=1`:

In [2]:
aluminium_setup(1)

PlaneWaveBasis discretization:
    architecture         : DFTK.CPU()
    num. mpi processes   : 1
    num. julia threads   : 1
    num. DFTK  threads   : 1
    num. blas  threads   : 2
    num. fft   threads   : 1

    Ecut                 : 7.0 Ha
    fft_size             : (24, 24, 24), 13824 total points
    kgrid                : MonkhorstPack([2, 2, 2])
    num.   red. kpoints  : 8
    num. irred. kpoints  : 8

    Discretized Model(lda_x+lda_c_pw, 3D):
        lattice (in Bohr)    : [7.65339   , 0         , 0         ]
                               [0         , 7.65339   , 0         ]
                               [0         , 0         , 7.65339   ]
        unit cell volume     : 448.29 Bohr³
    
        atoms                : Al₄
        pseudopot. family    : PseudoFamily("dojo.nc.sr.lda.v0_4_1.standard.upf")
    
        num. electrons       : 12
        spin polarization    : none
        temperature          : 0.001 Ha
        smearing             : DFTK.Smearing.FermiDi

and 5-fold as large supercell with `repeat=5`:

In [3]:
aluminium_setup(5)

PlaneWaveBasis discretization:
    architecture         : DFTK.CPU()
    num. mpi processes   : 1
    num. julia threads   : 1
    num. DFTK  threads   : 1
    num. blas  threads   : 2
    num. fft   threads   : 1

    Ecut                 : 7.0 Ha
    fft_size             : (96, 24, 24), 55296 total points
    kgrid                : MonkhorstPack([2, 2, 2])
    num.   red. kpoints  : 8
    num. irred. kpoints  : 8

    Discretized Model(lda_x+lda_c_pw, 3D):
        lattice (in Bohr)    : [38.267    , 0         , 0         ]
                               [0         , 7.65339   , 0         ]
                               [0         , 0         , 7.65339   ]
        unit cell volume     : 2241.5 Bohr³
    
        atoms                : Al₂₀
        pseudopot. family    : PseudoFamily("dojo.nc.sr.lda.v0_4_1.standard.upf")
    
        num. electrons       : 60
        spin polarization    : none
        temperature          : 0.001 Ha
        smearing             : DFTK.Smearing.FermiD

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [4]:
self_consistent_field(aluminium_setup(1); tol=1e-4);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -9.355197956956                   -1.10    6.0    158ms
  2   -9.356771307743       -2.80       -1.44    1.0   79.6ms
  3   -9.357073890295       -3.52       -2.78    1.2    121ms
  4   -9.357110868887       -4.43       -3.01   10.0    505ms
  5   -9.357111180869       -6.51       -3.15    1.0   75.4ms
  6   -9.357111344306       -6.79       -3.31    1.0   74.8ms
  7   -9.357111442014       -7.01       -3.47    1.0   76.0ms
  8   -9.357111478654       -7.44       -3.62    2.2   89.6ms
  9   -9.357111502580       -7.62       -3.92    1.0   74.2ms
 10   -9.357111507379       -8.32       -4.23    1.4   85.8ms


In [5]:
self_consistent_field(aluminium_setup(2); tol=1e-4);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -18.74777543990                   -0.97    6.5    489ms
┌ Warning: Eigensolver not converged
│   n_iter =
│    8-element Vector{Int64}:
│     1
│     1
│     1
│     7
│     1
│     9
│     1
│     1
└ @ DFTK ~/work/DFTK.jl/DFTK.jl/src/scf/self_consistent_field.jl:76
  2   -18.75913024969       -1.94       -1.32    2.8    290ms
  3   -18.79238192006       -1.48       -2.20    5.5    356ms
  4   -18.79259806331       -3.67       -2.52    5.8    419ms
  5   -18.79260506084       -5.16       -3.25    3.2    298ms
  6   -18.79260633718       -5.89       -3.65    4.8    416ms
  7   -18.79260638891       -7.29       -4.00    1.6    237ms


In [6]:
self_consistent_field(aluminium_setup(4); tol=1e-4);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -37.54961508755                   -0.84    8.8    1.86s
  2   -37.55788652842       -2.08       -1.23    3.5    801ms
  3   -37.56165927806       -2.42       -2.15    7.5    1.19s
  4   -37.56488522883       -2.49       -2.22   14.2    1.86s
  5   -37.56492546330       -4.40       -2.39    1.2    785ms
  6   -37.56494538546       -4.70       -3.09    2.1    802ms
  7   -37.56494837273       -5.52       -3.26   10.8    1.44s
  8   -37.56494972543       -5.87       -3.83    1.6    749ms
  9   -37.56494986854       -6.84       -4.42    4.8    1.40s


When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [7]:
self_consistent_field(aluminium_setup(1); tol=1e-4, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -9.355258821712                   -1.09    6.1    179ms
  2   -9.356836279100       -2.80       -1.91    1.0   67.3ms
  3   -9.357089240081       -3.60       -2.63    5.6    143ms
  4   -9.357088882350   +   -6.45       -2.57    3.4    115ms
  5   -9.357111049011       -4.65       -3.47    1.0   66.7ms
  6   -9.357111416398       -6.43       -3.84    5.1    139ms
  7   -9.357111502156       -7.07       -4.37    1.9   84.6ms


In [8]:
self_consistent_field(aluminium_setup(4); tol=1e-4, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -37.54617538489                   -0.84    9.6    1.96s
  2   -37.53489373929   +   -1.95       -1.56    3.1    690ms
  3   -17.85344057016   +    1.29       -0.42    9.4    1.72s
  4   -37.43516802129        1.29       -1.43    8.1    1.88s
  5   -37.54236655970       -0.97       -1.78    3.1    904ms
  6   -37.39320106969   +   -0.83       -1.44    5.2    1.04s
  7   -37.52465172872       -0.88       -1.75    4.6    1.02s
  8   -37.56433212575       -1.40       -2.33    2.9    824ms
  9   -37.56214761143   +   -2.66       -2.15    3.0    1.04s
 10   -37.56468160954       -2.60       -2.56    2.8    1.19s
 11   -37.56489856842       -3.66       -3.04    1.8    769ms
 12   -37.56493981040       -4.38       -3.42    3.2    1.07s
 13   -37.56494839544       -5.07       -3.70    3.6    965ms
 14   -37.56494971311       -5.88       -4.02    3.1    997ms


For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.